In [123]:
import numpy as np
import pandas as pd
import spacy

In [124]:
nlp = spacy.load('en_core_web_sm')

In [125]:
########################### Data processing ###############################

In [126]:
raw_df = pd.read_csv('Data/Womens Clothing E-Commerce Reviews.csv')

In [127]:
raw_df.head(10)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comfortable,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,"Love this dress! it's sooo pretty. i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite. i bought a petite and am 5'8"". i love the length on me- hits just a little below the knee. would definitely be a true midi on someone who is truly petite.",5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,"I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c",3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is not for the very petite. i am just under 5 feet tall and usually wear a 0p in this brand. this dress was very pretty out of the package but its a lot of dress. the skirt is long and very full so it overwhelmed my small frame. not a stranger to alterations, shortening and narrowing the skirt would take away from the embellishment of the garment. i love the color and the idea of the style but it just did not work on me. i returned this dress.",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,"I aded this in my basket at hte last mintue to see what it would look like in person. (store pick up). i went with teh darkler color only because i am so pale :-) hte color is really gorgeous, and turns out it mathced everythiing i was trying on with it prefectly. it is a little baggy on me and hte xs is hte msallet size (bummer, no petite). i decided to jkeep it though, because as i said, it matvehd everything. my ejans, pants, and the 3 skirts i waas trying on (of which i ]kept all ) oops.",5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, and had a ton of stuff (as always) to try on and used this top to pair (skirts and pants). everything went with it. the color is really nice charcoal with shimmer, and went well with pencil skirts, flare pants, etc. my only compaint is it is a bit big, sleeves are long and it doesn't go in petite. also a bit loose for me, but no xxs... so i kept it and wil ldecide later since the light color is already sold out in hte smallest size...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it runs a little snug in bust so i ordered up a size. very flattering and feminine with the usual retailer flair for style.,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite to make sure the length wasn't too long. i typically wear an xs regular in retailer dresses. if you're less busty (34b cup or smaller), a s petite will fit you perfectly (snug, but not tight). i love that i could dress it up for a party, or down for work. i love that the tulle is longer then the fabric underneath.",5,1,0,General,Dresses,Dresses


In [128]:
# keeping only the columns needed

reviews_df = raw_df[['Clothing ID', 'Review Text', 'Rating']]

In [129]:
# ensuring that all variables have the right data type

reviews_df.dtypes

Clothing ID     int64
Review Text    object
Rating          int64
dtype: object

In [130]:
# rename the variables for convenience

reviews_df = reviews_df.rename(
    columns = {
        'Clothing ID': 'Clothing_ID',
        'Review Text': 'Review_Text'
    }
)

In [131]:
reviews_df = reviews_df.dropna()

In [132]:
# get the indices corresponding to the first 5 and last 5 rows

indices = np.hstack(
    (np.arange(5), np.arange(-5, 0))

)

indices

array([ 0,  1,  2,  3,  4, -5, -4, -3, -2, -1])

In [133]:
# determining the top 5 and bottom 5 observations, based on the number of reviews left, in addition to the average product rating. 
# The analysis will focus only on those, but can naturally be extended to all the other observations

In [134]:
count_obs_clothing_ID_df = (
    reviews_df
    .groupby('Clothing_ID')
    # compute the count of observations per product (clothing ID)
    .size()
    # select the top 5 and bottom 5 observations accordingly
    .sort_values(ascending=False)
    .iloc[indices]
    # set the data back as a dataframe
    .reset_index(drop=False, name='count_obs')

)

count_obs_clothing_ID_df

,Clothing_ID,count_obs
0,1078,987
1,862,778
2,1094,735
3,1081,561
4,872,519
5,483,1
6,484,1
7,485,1
8,487,1
9,610,1


In [135]:
(count_obs_clothing_ID_df['count_obs']==1).all() # in this data we only have 1 review per product so using the count of observations (in fine purchases) is not meaningful

False

In [136]:
ratings_df = (
    reviews_df
    .groupby('Clothing_ID')
    # compute the average rating per product (clothing ID)
    ['Rating'].mean()
    # select the top 5 and bottom 5 observations accordingly
    .sort_values(ascending=False)
    .iloc[indices]
    # set the data back as a dataframe
    .reset_index(drop=False, name='avg_rating')

)

ratings_df

,Clothing_ID,avg_rating
0,1205,5.0
1,449,5.0
2,474,5.0
3,472,5.0
4,471,5.0
5,348,1.0
6,285,1.0
7,421,1.0
8,1176,1.0
9,669,1.0


In [137]:
#################### NLP - Analysis ###################################

In [138]:
# drop unneeded columns

reviews_df = reviews_df.drop(columns='Rating')

In [139]:
reviews_df = reviews_df.loc[
    
    # keep only clothes with a match in ratings_df
    reviews_df['Clothing_ID'].isin(
        ratings_df['Clothing_ID'].to_numpy()
    )
    
    
]

In [140]:
reviews_df = reviews_df.assign(
    
    processed_review = (
        reviews_df['Review_Text']
        # create a Doc object from each string (review)
        .apply(
            lambda input_str: nlp(input_str)
        )
    
    )


)

reviews_df

,Clothing_ID,Review_Text,processed_review
3108,285,"It seemed to stay in place and did not require constant readjustment. however, i found the band to be extremely small and tight. it is not the most comfortable bra because of the way the band just presses into your skin.","(It, seemed, to, stay, in, place, and, did, not, require, constant, readjustment, ., however, ,, i, found, the, band, to, be, extremely, small, and, tight, ., it, is, not, the, most, comfortable, bra, because, of, the, way, the, band, just, presses, into, your, skin, .)"
8122,472,These are such beautiful cheerful shorts. the floral motif just makes me smile. they do have brocade on them which is nice and dressy. they are perfect for a night out or a day at the beach. i went tts on them. i purchased similar shorts last year in the brown color. very nice shorts to have in your wardrobe..,"(These, are, such, beautiful, cheerful, shorts, ., the, floral, motif, just, makes, me, smile, ., they, do, have, brocade, on, them, which, is, nice, and, dressy, ., they, are, perfect, for, a, night, out, or, a, day, at, the, beach, ., i, went, tts, on, them, ., i, purchased, similar, shorts, last, year, in, the, brown, color, ., very, nice, shorts, to, have, in, your, wardrobe, ..)"
8125,472,"I typically wear boring shorts. you know - khaki. i saw these in the store and had to try them on. true to size in the waist, not too short, and the leg fit was perfect (slim without being skin tight). the pattern is far from boring. i can't wait to wear them out.","(I, typically, wear, boring, shorts, ., you, know, -, khaki, ., i, saw, these, in, the, store, and, had, to, try, them, on, ., true, to, size, in, the, waist, ,, not, too, short, ,, and, the, leg, fit, was, perfect, (, slim, without, being, skin, tight, ), ., the, pattern, is, far, from, boring, ., i, ca, n't, wait, to, wear, them, out, .)"
8133,472,"Love shorts that i can dress down w/ a chambray top or that i dress up w/ heeled sandals and a blouse. this is perfect!\r\ni found them to run a tad big.\r\ni have thin legs but don't have a small waist at all. i took a 0 in them, i'm usually more like a 2 on bottom.","(Love, shorts, that, i, can, dress, down, w/, a, chambray, top, or, that, i, dress, up, w/, heeled, sandals, and, a, blouse, ., this, is, perfect, !, \r\n, i, found, them, to, run, a, tad, big, ., \r\n, i, have, thin, legs, but, do, n't, have, a, small, waist, at, all, ., i, took, a, 0, in, them, ,, i, 'm, usually, more, like, a, 2, on, bottom, .)"
8150,472,"First off, they are much prettier irl than in the photos online. they fit very large on me. i am usually a 8 or 10 but have hips so ordered the10. wrong decision. i needed the 8.","(First, off, ,, they, are, much, prettier, irl, than, in, the, photos, online, ., they, fit, very, large, on, me, ., i, am, usually, a, 8, or, 10, but, have, hips, so, ordered, the10, ., wrong, decision, ., i, needed, the, 8, .)"
8151,472,"These shorts are extremely cute! i usually have a hard time finding shorts i like, but when i saw these the floral and brocade was too nice not to try. the length is good and they are not tight on the thighs, but not too loose either. the material makes them friendly for work as well as outings and running about town. i got my usual size 8 and found that to work perfectly!","(These, shorts, are, extremely, cute, !, i, usually, have, a, hard, time, finding, shorts, i, like, ,, but, when, i, saw, these, the, floral, and, brocade, was, too, nice, not, to, try, ., the, length, is, good, and, they, are, not, tight, on, the, thighs, ,, but, not, too, loose, either, ., the, material, makes, them, friendly, for, work, as, well, as, outings, and, running, about, town, ., i, got, my, usual, size, 8, and, found, that, to, work, perfectly, !)"
8782,348,Top is completely see through. i cant imagine if it were to get wet what it would look like. it is a cute design but not practical at all.,"(Top, is, completely, see, through, ., i, ca, nt, imagine, if, it, w

In [141]:
def get_unique_adjectives(input_nlp):
    '''Returns the unique adjectives found in the customer reviews
    '''
    # Part-of-speech (POS) tag should correspond to an adjective
    
    return set(token.lemma_ for token in input_nlp if token.pos_=='ADJ')
      

In [142]:
# get the unique adjectives from each customer review

reviews_df = reviews_df.assign(
    
    processed_review = (
        reviews_df['processed_review']
        .apply(
            lambda input_nlp: get_unique_adjectives(input_nlp)
        )
    
    )
)

reviews_df

,Clothing_ID,Review_Text,processed_review
3108,285,"It seemed to stay in place and did not require constant readjustment. however, i found the band to be extremely small and tight. it is not the most comfortable bra because of the way the band just presses into your skin.","{small, constant, comfortable, tight}"
8122,472,These are such beautiful cheerful shorts. the floral motif just makes me smile. they do have brocade on them which is nice and dressy. they are perfect for a night out or a day at the beach. i went tts on them. i purchased similar shorts last year in the brown color. very nice shorts to have in your wardrobe..,"{beautiful, floral, last, such, similar, nice, cheerful, dressy, brown, perfect}"
8125,472,"I typically wear boring shorts. you know - khaki. i saw these in the store and had to try them on. true to size in the waist, not too short, and the leg fit was perfect (slim without being skin tight). the pattern is far from boring. i can't wait to wear them out.","{true, slim, short, boring, perfect, tight}"
8133,472,"Love shorts that i can dress down w/ a chambray top or that i dress up w/ heeled sandals and a blouse. this is perfect!\r\ni found them to run a tad big.\r\ni have thin legs but don't have a small waist at all. i took a 0 in them, i'm usually more like a 2 on bottom.","{small, chambray, big, thin, perfect}"
8150,472,"First off, they are much prettier irl than in the photos online. they fit very large on me. i am usually a 8 or 10 but have hips so ordered the10. wrong decision. i needed the 8.","{wrong, pretty, large}"
8151,472,"These shorts are extremely cute! i usually have a hard time finding shorts i like, but when i saw these the floral and brocade was too nice not to try. the length is good and they are not tight on the thighs, but not too loose either. the material makes them friendly for work as well as outings and running about town. i got my usual size 8 and found that to work perfectly!","{floral, hard, loose, usual, good, friendly, nice, cute, tight}"
8782,348,Top is completely see through. i cant imagine if it were to get wet what it would look like. it is a cute design but not practical at all.,"{cute, wet, top, practical}"
9274,1176,I wore this suit for the first time yesterday and the black inner lining is leaking and causing stains on the white sections. seriously bummed as this is a super lovely suit.,"{inner, lovely, first, black, white}"
12315,471,"Love the mara hoffman basketweave bikini bottoms in fractals red. i went to her website for the size chart and got these in a size 'm'. very good fit, just a bit cheekier than her classic cut bikini bottoms. i got them with the matching basketweave bikini top.\r\n.\r\ni'm 5'2"", currently 33-25-37 to 38 depending on where i measure. upper hipbones are 34""; fuller thighs and bottom, so i never know what to order. i think a size 's' would be too skimpy so i hope these hold up well in the waves. \r\n.\r\nco","{red, full, good, cheeky, 5'2, skimpy, classic, upper}"
13678,669,"These are light and airy and pretty. they also tore the third time i wore them and not on a seam - a little above the knee while i was lying in my bed.\r\n\r\nthey also do not slim the figure and the fabric is slightly see-through as it is so thin, therefore if you are looking for serious lounge pants - great - i would not leave the house in them... or turn over in bed in them.\r\n\r\nwhen i sit, the slits fall down around my thighs. my son complimented my dress the other day because the slits give way","{serious, light, other, thin, third, little}"


In [143]:
reviews_df = (
    
    reviews_df
    # group together all reviews (unique adjectives) related to the same 
    # product
    .groupby('Clothing_ID')
    ['processed_review']
    .apply(
        # get unique adjectives from all the sets linked to a clothing ID
        lambda input_series: 
        set(adjective for set_i in input_series for adjective in set_i)
    )
    .reset_index(drop=False)

)

In [144]:
# join datasets, in order to sort data to have first the top 5 group, then later the bottom 5 group based on the average product rating

reviews_ratings_df = pd.merge(
    
    reviews_df,
    
    ratings_df,
    
    how='inner',
    
    on='Clothing_ID'

)

reviews_ratings_df

,Clothing_ID,processed_review,avg_rating
0,285,"{small, tight, comfortable, constant}",1.0
1,348,"{top, cute, wet, practical}",1.0
2,421,"{rough, wide}",1.0
3,449,"{cute, comfortable, flattering}",5.0
4,471,"{red, full, good, cheeky, 5'2, skimpy, classic, upper}",5.0
5,472,"{chambray, last, such, nice, cheerful, pretty, boring, dressy, beautiful, wrong, hard, big, usual, cute, brown, small, floral, similar, friendly, slim, perfect, large, loose, thin, good, true, short, tight}",5.0
6,474,"{substantial, lovely, other, happy, roomy, large, perfect, little}",5.0
7,669,"{serious, light, other, thin, third, little}",1.0
8,1176,"{lovely, inner, first, black, white}",1.0
9,1205,"{second, great, beige, 12th, perfect, regular, normal, fine}",5.0


In [145]:
reviews_ratings_df = reviews_ratings_df.sort_values(by='avg_rating', 
                                                    ascending=False,
                                                    ignore_index=True)

In [146]:
# drop unneeded columns

reviews_ratings_df = reviews_ratings_df.drop(columns='avg_rating')

In [147]:
pd.set_option('display.max_colwidth', None) # avoid truncating results

reviews_ratings_df

,Clothing_ID,processed_review
0,449,"{cute, comfortable, flattering}"
1,471,"{red, full, good, cheeky, 5'2, skimpy, classic, upper}"
2,472,"{chambray, last, such, nice, cheerful, pretty, boring, dressy, beautiful, wrong, hard, big, usual, cute, brown, small, floral, similar, friendly, slim, perfect, large, loose, thin, good, true, short, tight}"
3,474,"{substantial, lovely, other, happy, roomy, large, perfect, little}"
4,1205,"{second, great, beige, 12th, perfect, regular, normal, fine}"
5,285,"{small, tight, comfortable, constant}"
6,348,"{top, cute, wet, practical}"
7,421,"{rough, wide}"
8,669,"{serious, light, other, thin, third, little}"
9,1176,"{lovely, inner, first, black, white}"


In [148]:
raw_df.loc[(raw_df['Clothing ID'] == 474) | (raw_df['Clothing ID'] == 669)]

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
13678,13678,669,21,Pretty but cheap,"These are light and airy and pretty. they also tore the third time i wore them and not on a seam - a little above the knee while i was lying in my bed.\r\n\r\nthey also do not slim the figure and the fabric is slightly see-through as it is so thin, therefore if you are looking for serious lounge pants - great - i would not leave the house in them... or turn over in bed in them.\r\n\r\nwhen i sit, the slits fall down around my thighs. my son complimented my dress the other day because the slits give way",1,0,6,General Petite,Intimate,Lounge
17921,17921,474,62,NaN,"Just received these this morning and so happy. cartonnier always makes a substantial quality product. might run a little large,depends how roomy you want them, lovely colors and perfect length. wish they came in other colors.",5,1,0,General,Bottoms,Shorts


- For the top 5 group:

- Overall, we tend to note certain attributes which clear attributes making each product successful can be seen (e.g. for product 474 which is a pair of shorts, the comfort aspect seems to take precedence.

    - Accordingly a line of products targeted toward similar customers may be formed (focusing on the comfort/feel good in one's own body), if such segment is large enough to be profitable
    - Other features may be leveraged e.g. new colors, to account for further needs (i.e. microsegmentation)
    
<br>
- For highly rated products with relatively only positive products, we may keep them as they are, and plan (add only for minor updates/variations given the seasons/trend (monitoring the sales volume + customer feedback over time)

-----------------------------

- For the bottom 5 group:

- We note again relatively clear reasons for why a product is poorly considered (e.g. product 669 is deemed too thin)

- Given the sales volume of such products and the need or not to keep such products strategically (e.g. barrage against competitors to protect from an attack which may expand to more successful products), we may drop those, as the cost of better serving customers (correcting problems) might not be offset by more sales/price (to be assessed using a scenario analysis for instance accounting for the demand for such changes).

- The reviews help to improve future products, to avoid repeating errors (i.e. customer orientation)

- Products which have many negative reviews should possibl be dropped as they may lead to other future dissatisfied customers

In [ ]:
##########################################################

In [154]:
# fit an LDA Latent Dirichlet Analysis to extract topics - topic modeling
# and plotting a word cloud per topic allowing using to understand what the topic means

# see https://www.analyticsvidhya.com/blog/2018/10/mining-online-reviews-topic-modeling-lda/

In [155]:
def process_text(input_nlp):
    
    output = []
    
    for token in input_nlp:
        #if token.is_alpha: #alphanummeric characters only (letters)
        if token.pos_=='ADJ': # keep only adjectives
            #if not token.is_stop: #remove stop words e.g. 'the'
            output.append(token.lemma_)
    
    #keep unique adjectives
    output = set(output)            
    
    return ' '.join(output)
        

In [156]:
# if LDA ends up giving inconsistent results we 
#simply use below


# using POS tagging get all the adjectives in a list


#then we concatenate those lists per clothing ID

#and for the top 5 we interpret
